In [170]:
import numpy as np
import random
import langchain
from langchain.llms.base import LLM
import json
import re
import sqlite3
from datetime import datetime

In [30]:
#分配角色
char_quota={"seer":1,"witch":1,"hunter":1,"villager":2,"werewolf":3}

In [31]:
numbers=list(range(8))
random.shuffle(numbers)
numbers

[6, 7, 0, 4, 1, 5, 2, 3]

In [32]:
char_dist={}
char_dist[numbers[0]]="seer"
char_dist[numbers[1]]="witch"
char_dist[numbers[2]]="hunter"
char_dist[numbers[3]]="villager"
char_dist[numbers[4]]="villager"
char_dist[numbers[5]]="werewolf"
char_dist[numbers[6]]="werewolf"
char_dist[numbers[7]]="werewolf"
char_dist

{6: 'seer',
 7: 'witch',
 0: 'hunter',
 4: 'villager',
 1: 'villager',
 5: 'werewolf',
 2: 'werewolf',
 3: 'werewolf'}

In [44]:
#存活状态确认
aliv_list=list(range(8))
def char_alive_check(aliv_list,char_dist):
    char_aliv=[]
    for i in aliv_list:
        char_aliv.append(char_dist[i])
    return set(char_aliv)
a=char_alive_check(aliv_list,char_dist)

In [28]:
#角色初始定义

In [151]:
#大模型agent定义
import random
from http import HTTPStatus
import dashscope
from dashscope import Generation  # 建议dashscope SDK 的版本 >= 1.14.0

class base_llm():
    def __init__(self,system_prompt):
        self.api_key="sk-aaea7be7ccc047969804242b55c3aa53"
        self.system_prompt=system_prompt
    def call_with_messages(self,prompt_info):
        messages = [{'role': 'system', 'content': self.system_prompt},
                    {'role': 'user', 'content': prompt_info}]
        dashscope.api_key=self.api_key
        response = Generation.call(model="qwen-max-0107",
                                   messages=messages,
                                   # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                   seed=random.randint(1, 10000),
                                   # 将输出设置为"message"格式
                                   result_format='message')
        if response.status_code == HTTPStatus.OK:
            return response.output.choices[0].message.content
        else:
            return ('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message))

In [152]:
a=base_llm(system_prompt="你是一个聪明的助理")
b=a.call_with_messages("世界上最高的山峰是什么")
b

'珠穆朗玛峰（Mount Everest）是世界上海拔最高的山峰，位于喜马拉雅山脉中，其海拔高度为8,848.86米（29,031.7英尺），这是2020年中国和尼泊尔共同宣布的最新测量数据。'

In [150]:
# b.output.choices[0].message.content
# def extract_content(response):
#     response=json.loads(response)
#     for choice in response.get('output', {}).get('choices', []):
#         if 'message' in choice and 'content' in choice['message']:
#             return choice['message']['content']
#     return None

# # 使用函数提取content
# content = extract_content(b)
# print(content)

'珠穆朗玛峰，位于喜马拉雅山脉中，是世界海拔最高的山峰，其海拔高度为8,848.86米（2020年最新测量数据）。'

In [102]:
game_intro='''《狼人杀》是一款社交推理游戏，游戏模拟了一个小村庄里村民与狼人之间的斗争。游戏目标根据不同角色有所不同，村民方需要找出并消灭所有狼人，狼人方则要尽可能消灭村民，直至狼人数量等于或多于村民数量。
根据你提供的角色设置，这里有1个预言家、1个女巫、1个猎人、2个村民和3个狼人。下面是这些角色的基本职能和游戏的简要规则：
- 预言家：每个夜晚可以查验一名玩家的身份，了解这名玩家是村民方还是狼人方。
- 女巫：拥有两瓶药水，一瓶可以救活当夜被狼人杀害的玩家，另一瓶可以在夜晚毒杀一名玩家。每瓶药水只能使用一次。
- 猎人：在被淘汰时（不论是白天被投票淘汰还是夜里被狼人杀害），可以开枪带走一名玩家。
- 村民：没有特殊能力，主要通过讨论和投票来识别狼人。
- 狼人：每个夜晚会选择杀害一名玩家，目标是消灭所有村民，获得游戏胜利。
游戏流程大致如下：
- 游戏开始时，每位玩家随机分配一个角色，并保密自己的身份。
- 游戏分为夜晚和白天两个阶段。
- 夜晚：所有玩家闭眼，游戏主持人（通常称为法官）指挥特定角色的玩家按顺序睁眼执行他们的能力。首先是狼人相互确认身份，然后选择一个玩家作为杀害目标。随后，预言家可以查验一名玩家的身份，女巫会被告知狼人的杀害目标，决定是否使用药水。
- 白天：所有玩家睁眼。如果有玩家在夜里被杀害，法官会宣布该玩家死亡（除非被女巫救活）。然后所有存活的玩家进行讨论，最后进行投票，票数最多的玩家被淘汰出局。如果猎人被淘汰，他可以使用他的技能。
- 游戏继续进行，直到村民方或狼人方达成胜利条件。村民方胜利条件是所有狼人被消灭，狼人方胜利条件是狼人数量等于或多于存活的非狼人玩家数量。
'''
char_intro_werewolf='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是狼人，游戏开始时你知道你的狼人队友身份，你不知道其他玩家的身份。
    你和你的狼人队友属于狼人阵营，你们的胜利条件是消灭所有的村民或者所有的神。你的特殊能力是和狼人队友在晚上讨论后消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚和狼队友讨论后投票杀掉一个玩家。
    2.白天发表言论，保护自己和狼人队友，让大家投票杀掉一个玩家。
'''
char_intro_seer='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是预言家，游戏开始时你不知道其他玩家的身份。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是可以在夜晚验明一个玩家的真实身份。
    因此在游戏中你的工作包括以下内容：
    1.通过你的预言家身份，在夜晚查验某个玩家的身份。
    2.在白天，把你得到的信息告诉大家，如果你查验身份的玩家属于人类阵营，通过发言保护这个玩家不被投票消灭；如果你查验身份的玩家是狼人，引导大家用可能的方式消灭他。
'''
char_intro_witch='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是女巫，游戏开始时你不知道其他玩家的身份。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一瓶起死回生的解药和一瓶杀人的毒药，你可以在夜晚决定是否用解药救下被狼人杀掉的玩家，或者用毒药杀掉一个玩家，注意一局游戏中两种药都只能用一次。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚，根据你对当前形式的判断，决定是否用解药救下某个被杀掉的玩家，或者是否用毒药杀掉某个玩家。
    2.在白天发表言论，根据你观察到的信息决定保护人类阵营的玩家，同时帮大家找到狼人并投票杀掉狼人玩家。
'''
char_intro_hunter='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是猎人，游戏开始时你不知道其他玩家的身份。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一把猎枪，当你以任意方式死亡时，你可以立即决定是否用猎枪消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
    2.当你死亡时，根据你的判断决定是否杀掉某个你认为一定是狼人的玩家，或者不开枪以免误伤人类阵营队友；
'''
char_intro_villager='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是村民，游戏开始时你不知道其他玩家的身份。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你没有任何特殊能力。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
'''

In [120]:
def char_info_initialize(char_dist):
    char_info_dic={}
    for i in char_dist.keys():
        if char_dist[i]=='villager':
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是村民。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='seer':
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是预言家。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='witch':
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是女巫。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='werewolf':
            werewolf_list=[key for key,value in char_dist.items() if value=='werewolf']
            werewolf_list.remove(i)
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是狼人，你的狼人队友包括"%(i)
            for j in werewolf_list:
                info+="%d号玩家、"%(j)
            info=info[:-1]+"。\n"
            char_info_dic[i]=info
    return char_info_dic

In [121]:
a=char_info_initialize(char_dist=char_dist)
a

{6: '《狼人杀》是一款社交推理游戏，游戏模拟了一个小村庄里村民与狼人之间的斗争。游戏目标根据不同角色有所不同，村民方需要找出并消灭所有狼人，狼人方则要尽可能消灭村民，直至狼人数量等于或多于村民数量。\n根据你提供的角色设置，这里有1个预言家、1个女巫、1个猎人、2个村民和3个狼人。下面是这些角色的基本职能和游戏的简要规则：\n- 预言家：每个夜晚可以查验一名玩家的身份，了解这名玩家是村民方还是狼人方。\n- 女巫：拥有两瓶药水，一瓶可以救活当夜被狼人杀害的玩家，另一瓶可以在夜晚毒杀一名玩家。每瓶药水只能使用一次。\n- 猎人：在被淘汰时（不论是白天被投票淘汰还是夜里被狼人杀害），可以开枪带走一名玩家。\n- 村民：没有特殊能力，主要通过讨论和投票来识别狼人。\n- 狼人：每个夜晚会选择杀害一名玩家，目标是消灭所有村民，获得游戏胜利。\n游戏流程大致如下：\n- 游戏开始时，每位玩家随机分配一个角色，并保密自己的身份。\n- 游戏分为夜晚和白天两个阶段。\n- 夜晚：所有玩家闭眼，游戏主持人（通常称为法官）指挥特定角色的玩家按顺序睁眼执行他们的能力。首先是狼人相互确认身份，然后选择一个玩家作为杀害目标。随后，预言家可以查验一名玩家的身份，女巫会被告知狼人的杀害目标，决定是否使用药水。\n- 白天：所有玩家睁眼。如果有玩家在夜里被杀害，法官会宣布该玩家死亡（除非被女巫救活）。然后所有存活的玩家进行讨论，最后进行投票，票数最多的玩家被淘汰出局。如果猎人被淘汰，他可以使用他的技能。\n- 游戏继续进行，直到村民方或狼人方达成胜利条件。村民方胜利条件是所有狼人被消灭，狼人方胜利条件是狼人数量等于或多于存活的非狼人玩家数量。\n你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神（预言家、女巫和猎人），这局中你抽到的角色是村民，游戏开始时你不知道其他玩家的身份。\n    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你没有任何特殊能力。\n    因此在游戏中你的工作包括以下内容：\n    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。\n你是6号玩家，你的身份是预言家。',
 7: '《狼人杀》是一款社交推理游戏，游戏模拟了一个小村庄里村民与狼人之间的斗争。游戏目标根据不同角色有所不同，村民方需要找

In [165]:
a="sdgsdhshf【猎枪决策】:sdgdfgsdfgdsf"
g=re.search(r"【猎枪决策】(.*)",a)
g.group(1)[1:]

'sdgdfgsdfgdsf'

In [29]:
#狼人
class werewolf():
    def __init__(self,base_llm,char_info):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
    def kill_speech(self,cur_state,strategy):     #杀人发言
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考应该消灭哪一个玩家，并在和其他狼人的讨论中发言。
        请把你的思考过程写到【思考过程】中，然后把发言内容写到【发言内容】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        return res
    def kill_vote(self,cur_state,strategy):     #杀人投票
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考，并决定投票消灭哪一个玩家。
        请把你的思考过程写到【思考过程】中，然后把投票结果写到【投票结果】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        return res
    def speech(self,cur_state,strategy):    #白天发言
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考如何发言才能获胜，并发表你的看法，误导人类阵营。
        请把你的思考过程写到【思考过程】中，然后把发言内容写到【发言内容】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        return res
    def day_vote(self,cur_state,strategy):    #白天投票
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请把你的思考过程写到【思考过程】中，然后把投票结果写到【投票结果】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        return res

In [35]:
#seer
class seer():
    def __init__(self,base_llm,char_info):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
    def char_val(self,cur_state,strategy,known_info):     #验身份
        prompt=cur_state+known_info+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考验证哪一个嫌疑最大的玩家。
        请把你的思考过程写到【思考过程】中，然后把验身份的对象玩家编号写到【玩家编号】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【玩家编号】(.*)",result).group(1)[1:]
        return res
    def speech(self,cur_state,strategy):    #白天发言
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请把你的思考过程写到【思考过程】中，然后把发言内容写到【发言内容】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        return res
    def day_vote(self,cur_state,strategy):    #白天投票
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请把你的思考过程写到【思考过程】中，然后把投票结果写到【投票结果】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        return res

In [36]:
#witch
class witch():
    def __init__(self,base_llm,char_info):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
    def use_poison(self,cur_state,strategy,killed_player):     #毒药
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考是否使用女巫的毒药消灭一名玩家，以及消灭哪位玩家。
        请把你的思考过程写到【思考过程】中，然后把你的决定写到【毒药决策】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【毒药决策】(.*)",result).group(1)[1:]
        return res
    def use_antidote(self,cur_state,strategy,killed_player):     #解药
        prompt=cur_state+strategy+'''现在是晚上时间,%d号玩家被狼人消灭,请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考，并是否使用解药复活他。
        请把你的思考过程写到【思考过程】中，然后把你的决定写到【解药决策】中。
        【思考过程】:'''%（killed_player）
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【解药决策】(.*)",result).group(1)[1:]
        return res
    def speech(self,cur_state,strategy):    #发言
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请把你的思考过程写到【思考过程】中，然后把发言内容写到【发言内容】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        return res
    def day_vote(self,cur_state,strategy):    #白天投票
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请把你的思考过程写到【思考过程】中，然后把投票结果写到【投票结果】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        return res

In [34]:
#villager
class villager():
    def __init__(self,base_llm,char_info):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
    def speech(self,cur_state,strategy):    #发言
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请把你的思考过程写到【思考过程】中，然后把发言内容写到【发言内容】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        return res
    def day_vote(self,cur_state,strategy):    #白天投票
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请把你的思考过程写到【思考过程】中，然后把投票结果写到【投票结果】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        return res

In [37]:
#huter
class huter():
    def __init__(self,base_llm,char_info):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
    def vengence(self,cur_state,strategy):     #复仇子弹
        prompt=cur_state+strategy+'''现在你即将被消灭，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考是否使用猎人的猎枪复仇消灭一名玩家，以及消灭哪位玩家。
        请把你的思考过程写到【思考过程】中，然后把你的决定写到【猎枪决策】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【猎枪决策】(.*)",result).group(1)[1:]
        return res
    def speech(self,cur_state,strategy):    #发言
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请把你的思考过程写到【思考过程】中，然后把发言内容写到【发言内容】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        return res
    def day_vote(self,cur_state,strategy):    #白天投票
        prompt=cur_state+strategy+'''现在是白天投票时间，请你根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭哪个玩家。
        请把你的思考过程写到【思考过程】中，然后把投票结果写到【投票结果】中。
        【思考过程】:'''
        result=self.llm.call_with_messages(prompt)
        res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        return res

In [54]:
#game_endcheck
def endcheck(aliv_list,char_dist):
    char_aliv=char_alive_check(aliv_list,char_dist)
    if "werewolf" not in char_aliv:
        return 2
    elif "villager" not in char_aliv:
        return 1
    elif char_aliv=={"werewolf","villager"}:
        return 1
    else:
        return 0    
endcheck(aliv_list,char_dist)

0

In [157]:
a='''{'是否消灭':'是','消灭玩家':3}'''
b=eval(a)
b['消灭玩家']

3

In [160]:
#动作信息提取
input_2_json_llm=base_llm("你是一个从对话内容提取json字符串的小助手，可以根据提示从对话中精准提取信息，并以规定的json格式返回")
def kill_vote_extr(base_llm,input):
    prompt='''请根据狼人的投票内容提取他的投票消灭的对象，并且以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}
    例1：狼人的【投票结果】:根据当前的局面，我认为消灭3号玩家最好，因为他比较像预言家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：狼人的【投票结果】:2号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':2}
    #####################################
    当前狼人的【投票结果】:%s。
    【提取结果】:
    '''%(input)
    result=self.llm.call_with_messages(prompt)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def day_vote_extr(base_llm,input):
    prompt='''请根据当前玩家的投票内容提取他的投票消灭的对象，并且以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}
    例1：狼人的【投票结果】:根据当前的局面，我认为消灭3号玩家最好。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：狼人的【投票结果】:2号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':2}
    #####################################
    当前狼人的【投票结果】:%s。
    【提取结果】:
    '''%(input)
    result=self.llm.call_with_messages(prompt)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def seer_val_extr(base_llm,input):
    prompt='''请根据玩家的选择决策提取他的投票验证身份的对象，并且以类似以下格式返回结果:{'是否验证':'是','验证玩家':4}
    例1：玩家的【选择决策】:根据当前的局面，我想知道3号玩家的角色是什么。
    【提取结果】:{'是否验证':'是','验证玩家':3}
    例2：玩家的【选择决策】:2号玩家。
    【提取结果】:{'是否验证':'是','验证玩家':3}
    #####################################
    当前玩家的【选择决策】:%s。
    【提取结果】:
    '''%(input)
    result=self.llm.call_with_messages(prompt)
    kill_res=eval(result)
    if kill_res['是否验证']=='是':
        return kill_res['验证玩家']
    else:
        return -1
def witch_poison_extr(base_llm,input):
    prompt='''请根据当前玩家的毒药决策提取他的决定用毒药消灭的对象，并且以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}
    例1：玩家的【毒药决策】:根据当前的局面，我想用毒药毒死3号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：玩家的【毒药决策】:2号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':2}
    #####################################
    当前玩家的【毒药决策】:%s。
    【提取结果】:
    '''%(input)
    result=self.llm.call_with_messages(prompt)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def witch_antidote_extr(base_llm,input):
    prompt='''请根据当前玩家的解药决策提取是否解救的决定，并且以类似以下格式返回结果:{'是否解救':'是'}
    例1：玩家的【解药决策】:根据当前的局面，我不想用解药救下这个人。
    【提取结果】:{'是否解救':'否'}
    例2：玩家的【解药决策】:救。
    【提取结果】:{'是否解救':'是'}
    #####################################
    当前玩家的【解药决策】:%s。
    【提取结果】:
    '''%(input)
    result=self.llm.call_with_messages(prompt)
    kill_res=eval(result)
    return kill_res['是否解救']
def hunter_vegence_extr(base_llm,input):
    prompt='''请根据当前玩家的猎枪决策提取他的决定用猎枪复仇消灭的对象，并且以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}
    例1：玩家的【猎枪决策】:根据当前的局面，我想用带走3号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：玩家的【猎枪决策】:2号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':2}
    #####################################
    当前玩家的【猎枪决策】:%s。
    【提取结果】:
    '''%(input)
    result=self.llm.call_with_messages(prompt)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1

In [169]:
#建立数据库
conn = sqlite3.connect('werewolf.db')

In [ ]:
class game_logger():
    def __init__(self,db_name):
        self.conn=sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.table_name=datetime.now().strftime("%Y%m%d%H%M%S")
        create_table_sql = f'''CREATE TABLE IF NOT EXISTS {self.table_name} (
        id INTEGER PRIMARY KEY,
        data TEXT NOT NULL
            )
        '''
        self.cursor.execute(create_table_sql)
    def 

In [ ]:
#night process
def night(game_log,alive_list,char_dist):
    pass

In [ ]:
#day process
def night(game_log,alive_list,char_dist):
    pass